In [ ]:
from functools import partial
import os
from pathlib import Path

from dustgoggles.func import zero
from killscreen.utilities import roundstring

# hacky; can remove if we decide to add an install script or put this in the repo root
os.chdir(globals()['_dh'][0].parent)

from subset.benchmark.bench_utils import (
    check_existing_benchmarks,
    dump_bandwidth_allowance_metrics,
    dump_throwaway_results
)
from subset.benchmark.handlers import (
    execute_test_case, interpret_benchmark_instructions, process_bench_stats
)
from subset.utilz.generic import summarize_stat
from subset.utilz.mount_s3 import mount_bucket

simple execution environment for dataset benchmarking. executes benchmarks defined in the benchmark_settings module.

important notes:
* the `fsspec`-based "s3" and "s3_section" loaders will only function on the barentsen/cloud-support astropy branch.
* other loaders will only function if `goofys` is executable from the user's path.
* attempts to access private buckets, via either `goofys` or `fsspec`, will use credentials stored in ~/.aws/credentials.

In [ ]:
SETTINGS = {
    # max files: tests will use the top n_files elements of the TEST_FILES
    # attribute of each benchmark module (unless less than n_files are given
    # in TEST_FILES)
    "n_files": 25,
    # do we actually want to look at the cuts? probably not in big bulk
    # benchmarks, but can be useful for diagnostics or if you want a
    # screensaver or whatever.
    "return_cuts": False,
    # rng seed for consistent execution w/out having to explicitly define
    # a very long list of rectangles
    "seed": 123456,
    # note that bandwidth throttling is not available in the TIKE environment.
}

# these correspond to the names of submodules of the benchmark_settings module
BENCHMARK_NAMES = (
    "hst", "panstarrs", "galex_rice", "jwst_crf", "tesscut", "galex_gzip",
    "hst_big", "spitzer_irac", "spitzer_cosmos_irac"
)
# if there is an existing result file corresponding to a particular
# test case, shall we run it again (w/incrementing suffixes attached to
# outputs?)
DUPLICATE_BENCHMARKS = True
# where shall we write benchmark results?
METRIC_DIRECTORY = "subset/benchmark/bench_results"
Path(METRIC_DIRECTORY).mkdir(parents=True, exist_ok=True)
# running throwaway tests using a FUSE loader completely cheeses
# the statistics without remount. TODO item to run the throwaways
# with an fsspec-backed loader here.
N_THROWAWAYS = 0

# "benchmarks" contains instructions for each test case
benchmarks = {
    name: interpret_benchmark_instructions(name, SETTINGS)
    for name in BENCHMARK_NAMES
}
# TIKE has pre-established per-bucket goofys mountpoints, which we squeeze
# into the benchmark definitions
mountpoints = {'stpubdata': "/s3/gf/stpubdata", 'nishapur': '/s3/gf/nishapur'}
for name, benchmark in benchmarks.items():
    for case in benchmark:
        if case['bucket'] is not None:
            case['paths'] = [str(Path(mountpoints[case['bucket']], p)) for p in case['paths']]
            
# we cannot dynamically remount buckets on TIKE, so we pass a dummy 
# mount function to the handler.
remount = zero

In [ ]:
# filter repetitive benchmarks
for name, benchmark in benchmarks.items():
    deletions = []
    for ix, case in enumerate(benchmark):
        if (
            'astropy' not in case['title']
            or 'greedy' in case['title']
            or 'preload' in case['title']
            or case['count'] != 5
            or case['shape'] != (40, 40)
            or 'panstarrs' not in case['title']
        ):
            deletions.append(ix)
    benchmarks[name] = [
        benchmark[ix] for ix in range(len(benchmark)) if ix not in deletions
    ]
        

In [ ]:
"""
execute all benchmarks in a loop and save results. you can mess with this cell
to pretty straightforwardly look at only certain categories of cases,
suppress results, etc.
"""
logs = {}
for benchmark_name, benchmark in benchmarks.items():
    previous = {}
    for case in benchmark:
        skip, suffix = check_existing_benchmarks(
            case['title'], DUPLICATE_BENCHMARKS, METRIC_DIRECTORY
        )
        if skip is True:
            continue
        cuts, stat, log, throwaways = execute_test_case(
            case, previous, remount, N_THROWAWAYS
        )
        print(roundstring(summarize_stat(stat)) + "\n")
        logs[f"{case['title']}"] = log
        process_bench_stats(log, case, benchmark_name).to_csv(
            Path(METRIC_DIRECTORY, f"{case['title']}_benchmark_{suffix}.csv"),
            index=None
        )
        # note that we can't dump bandwidth allowance metrics on TIKE
        dump_throwaway_results(throwaways, case['title'], suffix, METRIC_DIRECTORY)
        previous = case
